In [20]:
import numpy as np
import pandas as pd
import psycopg2 as psy
import plotly.express as px
import geopandas as gpd
from sshtunnel import SSHTunnelForwarder
from sqlalchemy import create_engine
from plotly.subplots import make_subplots

In [21]:
import warnings
warnings.filterwarnings("ignore")

In [22]:
pin_query_today = '''
with base as (
    select      ops_main.awb
    ,           date_trunc('day',ops_main.first_ofd_time) as first_ofd_date
    ,           date_trunc('day',ops_main.delivertime) as delivered_date
    ,           ops_main.delivertime
    ,           ops_main.shipping_city
    ,           ops_main.edd
    ,           ops_main.shipping_pincode
    ,           substring(ops_main.sort_code from 14) as cluster_code
    ,           "public"."locus_task_brief"."status" as task_status
    ,           "public"."locus_task_brief"."dispatch_time" as locus_dispatch_time
    ,           RANK() OVER ( PARTITION BY locus_task_brief.awb ORDER BY locus_task_brief.dispatch_time) as attempt_number
    from        ops_main
    left join "public"."locus_task_brief" on ops_main."awb" = "public"."locus_task_brief"."awb"
    left join "public"."locus_tour_brief" on "public"."locus_task_brief"."tour_id" = "public"."locus_tour_brief"."tour_id"
    where       1=1
    and         shipping_partner = 'Hyperlocal')
, final_raw as (
    select      *
    ,           case when delivered_date <= edd  then 1 else 0 end as otd
    ,           case when first_ofd_date <= edd then 1 else 0 end as ofd
    from        base
)
, otd_final as (
    select      
                shipping_city
    ,           cluster_code
    ,           shipping_pincode
    ,           100.0*count(distinct(case when otd = 1 then awb else null end))/nullif(count(distinct(awb)),0) as otd
    ,           100.0*count(distinct(case when ofd = 1 then awb else null end))/nullif(count(distinct(awb)),0) as ofd
    from final_raw
    where       1=1
    and         date_trunc('day',edd) = date_trunc('day', now() + interval '5.5 hours' - interval '1 day')
    group by    1,2,3
    order by    1,2,3 desc)
, fasr_final as (
    select      
                shipping_city
    ,           cluster_code
    ,           shipping_pincode
    ,           100.0*sum(case when attempt_number=1 and task_status = 'COMPLETED'  then 1 else 0 end)/nullif(sum(case when attempt_number = 1 then 1 else 0 end),0) as fasr
    from final_raw
    where       1=1
    and         date_trunc('day',locus_dispatch_time) = date_trunc('day', now() + interval '5.5 hours' - interval '1 day')
    group by    1,2,3
    order by    1,2,3 desc)
select      otd_final.shipping_city, otd_final.cluster_code, otd_final.shipping_pincode, otd, ofd, fasr
from        otd_final
left join   fasr_final
on otd_final.cluster_code = fasr_final.cluster_code and otd_final.shipping_pincode = fasr_final.shipping_pincode and otd_final.shipping_city=fasr_final.shipping_city
'''
cluster_query_today = '''
with base as (
    select      ops_main.awb
    ,           date_trunc('day',ops_main.first_ofd_time) as first_ofd_date
    ,           date_trunc('day',ops_main.delivertime) as delivered_date
    ,           ops_main.delivertime
    ,           ops_main.shipping_city
    ,           ops_main.edd
    ,           ops_main.shipping_pincode
    ,           substring(ops_main.sort_code from 14) as cluster_code
    ,           "public"."locus_task_brief"."status" as task_status
    ,           "public"."locus_task_brief"."dispatch_time" as locus_dispatch_time
    ,           RANK() OVER ( PARTITION BY locus_task_brief.awb ORDER BY locus_task_brief.dispatch_time) as attempt_number
    from        ops_main
    left join "public"."locus_task_brief" on ops_main."awb" = "public"."locus_task_brief"."awb"
    left join "public"."locus_tour_brief" on "public"."locus_task_brief"."tour_id" = "public"."locus_tour_brief"."tour_id"
    where       1=1
    and         shipping_partner = 'Hyperlocal')
, final_raw as (
    select      *
    ,           case when delivered_date <= edd  then 1 else 0 end as otd
    ,           case when first_ofd_date <= edd then 1 else 0 end as ofd
    from        base)
, otd_final as (
    select      
                shipping_city
    ,           cluster_code
    ,           100.0*count(distinct(case when otd = 1 then awb else null end))/nullif(count(distinct(awb)),0) as otd
    ,           100.0*count(distinct(case when ofd = 1 then awb else null end))/nullif(count(distinct(awb)),0) as ofd
    from final_raw
    where       1=1
    and         date_trunc('day',edd) = date_trunc('day', now() + interval '5.5 hours' - interval '1 day')
    group by    1,2
    order by    1,2 desc)
, fasr_final as (
    select      
                shipping_city
    ,           cluster_code
    ,           100.0*sum(case when attempt_number=1 and task_status = 'COMPLETED'  then 1 else 0 end)/nullif(sum(case when attempt_number = 1 then 1 else 0 end),0) as fasr
    from final_raw
    where       1=1
    and         date_trunc('day',locus_dispatch_time) = date_trunc('day', now() + interval '5.5 hours' - interval '1 day')
    group by    1,2
    order by    1,2 desc)
select      otd_final.shipping_city, otd_final.cluster_code, otd, ofd, fasr
from        otd_final
left join   fasr_final on otd_final.cluster_code = fasr_final.cluster_code and otd_final.shipping_city=fasr_final.shipping_city'''
pin_query_week = '''
with base as (
    select      ops_main.awb
    ,           date_trunc('day',ops_main.first_ofd_time) as first_ofd_date
    ,           date_trunc('day',ops_main.delivertime) as delivered_date
    ,           ops_main.delivertime
    ,           ops_main.shipping_city
    ,           ops_main.edd
    ,           ops_main.shipping_pincode
    ,           substring(ops_main.sort_code from 14) as cluster_code
    ,           "public"."locus_task_brief"."status" as task_status
    ,           "public"."locus_task_brief"."dispatch_time" as locus_dispatch_time
    ,           RANK() OVER ( PARTITION BY locus_task_brief.awb ORDER BY locus_task_brief.dispatch_time) as attempt_number
    from        ops_main
    left join "public"."locus_task_brief" on ops_main."awb" = "public"."locus_task_brief"."awb"
    left join "public"."locus_tour_brief" on "public"."locus_task_brief"."tour_id" = "public"."locus_tour_brief"."tour_id"
    where       1=1
    and         shipping_partner = 'Hyperlocal')
, final_raw as (
    select      *
    ,           case when delivered_date <= edd  then 1 else 0 end as otd
    ,           case when first_ofd_date <= edd then 1 else 0 end as ofd
    from        base
    )
, otd_final as (
    select      
                shipping_city
    ,           cluster_code
    ,           shipping_pincode
    ,           100.0*count(distinct(case when otd = 1 then awb else null end))/nullif(count(distinct(awb)),0) as otd
    ,           100.0*count(distinct(case when ofd = 1 then awb else null end))/nullif(count(distinct(awb)),0) as ofd
    from final_raw
    where       1=1
    and         date_trunc('week',edd) = date_trunc('week', now() + interval '5.5 hours') - interval '1 week'
    group by    1,2,3
    order by    1,2,3 desc)
, fasr_final as (
    select      
                shipping_city
    ,           cluster_code
    ,           shipping_pincode
    ,           100.0*sum(case when attempt_number=1 and task_status = 'COMPLETED'  then 1 else 0 end)/nullif(sum(case when attempt_number = 1 then 1 else 0 end),0) as fasr
    from final_raw
    where       1=1
    and         date_trunc('week',locus_dispatch_time) = date_trunc('week', now() + interval '5.5 hours') - interval '1 week'
    group by    1,2,3
    order by    1,2,3 desc)
select      otd_final.shipping_city, otd_final.cluster_code, otd_final.shipping_pincode, otd, ofd, fasr
from        otd_final
left join   fasr_final on otd_final.cluster_code = fasr_final.cluster_code and otd_final.shipping_pincode = fasr_final.shipping_pincode and otd_final.shipping_city=fasr_final.shipping_city
'''
cluster_query_week = '''
with base as (
    select      ops_main.awb
    ,           date_trunc('day',ops_main.first_ofd_time) as first_ofd_date
    ,           date_trunc('day',ops_main.delivertime) as delivered_date
    ,           ops_main.delivertime
    ,           ops_main.shipping_city
    ,           ops_main.edd
    ,           ops_main.shipping_pincode
    ,           substring(ops_main.sort_code from 14) as cluster_code
    ,           "public"."locus_task_brief"."status" as task_status
    ,           "public"."locus_task_brief"."dispatch_time" as locus_dispatch_time
    ,           RANK() OVER ( PARTITION BY locus_task_brief.awb ORDER BY locus_task_brief.dispatch_time) as attempt_number
    from        ops_main
    left join "public"."locus_task_brief" on ops_main."awb" = "public"."locus_task_brief"."awb"
    left join "public"."locus_tour_brief" on "public"."locus_task_brief"."tour_id" = "public"."locus_tour_brief"."tour_id"
    where       1=1
    and         shipping_partner = 'Hyperlocal')
, final_raw as (
    select      *
    ,           case when delivered_date <= edd  then 1 else 0 end as otd
    ,           case when first_ofd_date <= edd then 1 else 0 end as ofd
    from        base)
, otd_final as (
    select      
                shipping_city
    ,           cluster_code
    ,           100.0*count(distinct(case when otd = 1 then awb else null end))/nullif(count(distinct(awb)),0) as otd
    ,           100.0*count(distinct(case when ofd = 1 then awb else null end))/nullif(count(distinct(awb)),0) as ofd
    from final_raw
    where       1=1
    and         date_trunc('week',edd) = date_trunc('week', now() + interval '5.5 hours') - interval '1 week'
    group by    1,2
    order by    1,2 desc)
, fasr_final as (
    select      
                shipping_city
    ,           cluster_code
    ,           100.0*sum(case when attempt_number=1 and task_status = 'COMPLETED'  then 1 else 0 end)/nullif(sum(case when attempt_number = 1 then 1 else 0 end),0) as fasr
    from final_raw
    where       1=1
    and         date_trunc('week',locus_dispatch_time) = date_trunc('week', now() + interval '5.5 hours') - interval '1 week'
    group by    1,2
    order by    1,2 desc)
select      otd_final.shipping_city, otd_final.cluster_code, otd, ofd, fasr
from        otd_final
left join   fasr_final on otd_final.cluster_code = fasr_final.cluster_code and otd_final.shipping_city=fasr_final.shipping_city
'''

In [23]:
def plot_pincodes_with_hubs(df_op, title, column, cols_list, zoom_level, df_cluster = None):
    lat_center = df_op['geometry'].centroid.apply(lambda x: x.xy[1][0]).mean()
    long_center = df_op['geometry'].centroid.apply(lambda x: x.xy[0][0]).mean()
    fig = px.choropleth_mapbox(df_op,
                           geojson=df_op.geometry,
                           locations=df_op.index,
                           color=column,
                           hover_data = cols_list,
                           color_continuous_scale = ["Red", "Yellow", "Green"],
                           center={
                               "lat": lat_center,
                               "lon": long_center
                               },
                           mapbox_style="open-street-map",
                           opacity=0.6,
                           zoom=zoom_level,
                           height=800,
                           width=800,
                           title=title)
    if df_cluster is not None:
      clusters = px.choropleth_mapbox(df_cluster,
                           geojson=df_cluster.geometry,
                           locations=df_cluster.index,
                           mapbox_style="open-street-map",
                           opacity=0.1)
      clusters.update_traces(
      marker_line_width=5)
      fig.add_traces(list(clusters.select_traces()))

    return fig

In [24]:
#GeoJSON Conversion
df_shp = gpd.read_file('India_Pincodes/india_pincodes.shp')
df_shp['area'] = df_shp['geometry'].to_crs({'init': 'epsg:3395'}).map(lambda p: p.area / 10**6)
df_shp['pincode'] = df_shp['pincode'].astype(int).astype(str)
df_shp.head()

,pincode,state,district,officename,officetype,geometry,area
0,321023,Rajasthan,Bharatpur,Jurhera,S.O,"POLYGON ((77.13926 27.76213, 77.13982 27.76237...",163.179283
1,322236,Rajasthan,Karauli,Dhindora,S.O,"POLYGON ((77.07143 26.84008, 77.06852 26.83751...",195.671285
2,321024,Rajasthan,Bharatpur,Sikri,S.O,"POLYGON ((76.93663 27.60967, 76.93636 27.61241...",350.382224
3,322252,Rajasthan,Karauli,Suroth,S.O,"POLYGON ((77.13675 26.80514, 77.13852 26.81275...",153.992396
4,321201,Rajasthan,Bharatpur,Kumher,S.O,"POLYGON ((77.20875 27.33833, 77.20870 27.34305...",209.093186


In [25]:
SSH_requiered = 'No'
db='datawarehouse'
DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
conn = []
if SSH_requiered == 'Yes':
  SSH_HOST='ec2-3-111-187-137.ap-south-1.compute.amazonaws.com'
  #LOCALHOST="0.0.0.0"
  ssh_tunnel= SSHTunnelForwarder(
          (SSH_HOST),
          ssh_username="ec2-user",
          ssh_private_key= 'assets/tunnel-ssh.cer',
          ssh_private_key_password= "",
          remote_bind_address=(DB_HOST, 5432))
  #ssh_tunnel._server_list[0].block_on_close = False
  ssh_tunnel.start()
  conn = psy.connect(
      host=ssh_tunnel.local_bind_host,
      port=ssh_tunnel.local_bind_port,
      user='postgres',
      password= "Simply1234",
      database='postgres')
else:
  conn = psy.connect(
      host = DB_HOST,
      port = 5432,
      user = 'postgres',
      password= "Simply1234",
      database='postgres')

city_list = ["Bangalore", "Delhi", "Mumbai", "Hyderabad"]

pincode_perf_df_today = pd.read_sql(pin_query_today, conn)
# print(pincode_perf_df_today.head())

cluster_perf_df_today = pd.read_sql(cluster_query_today, conn)
# print(cluster_perf_df_today.head())

pincode_perf_df_week = pd.read_sql(pin_query_week, conn)
# print(pincode_perf_df_week.head())

cluster_perf_df_week = pd.read_sql(cluster_query_week, conn)
# print(cluster_perf_df_week.head())


for city in city_list:


  pincode_perf_df_today_temp = pincode_perf_df_today[pincode_perf_df_today['shipping_city']== city].drop(['shipping_city'], axis = 1)
  cluster_perf_df_today_temp = cluster_perf_df_today[cluster_perf_df_today['shipping_city']== city].drop(['shipping_city'], axis = 1)
  pincode_perf_df_week_temp = pincode_perf_df_week[pincode_perf_df_week['shipping_city']== city].drop(['shipping_city'], axis = 1)
  cluster_perf_df_week_temp = cluster_perf_df_week[cluster_perf_df_week['shipping_city']== city].drop(['shipping_city'], axis = 1)
#  print(cluster_perf_df_today.head())
#  print(cluster_perf_df_week.head())

  column_rename_mapping_1= {
    'shipping_pincode': 'pincode',
    'cluster_code': 'Cluster',
    'otd': 'OTD',
    'ofd': 'OFD',
    'fasr': 'FASR'
  }
  column_rename_mapping_2= {
    'cluster_code': 'Cluster',
    'otd': 'OTD',
    'ofd': 'OFD',
    'fasr': 'FASR'
  }
  pincode_perf_df_today_temp = pincode_perf_df_today_temp.rename(columns=column_rename_mapping_1)

  pincode_perf_df_today_temp['pincode'] = pincode_perf_df_today_temp['pincode'].astype(int).astype(str)
  pincode_df_today = df_shp.merge(pincode_perf_df_today_temp, how='inner', on='pincode')
  pincode_df_today = pincode_df_today.set_index('pincode')

  cluster_perf_df_today_temp = cluster_perf_df_today_temp.rename(columns=column_rename_mapping_2)

  df_cluster_today = pincode_df_today[['Cluster', 'geometry']].dissolve(by='Cluster')
  df_cluster_today.reset_index(inplace = True)
  cluster_df_today = df_cluster_today.set_index('Cluster').join(cluster_perf_df_today_temp.set_index('Cluster'))

  pincode_perf_df_week_temp = pincode_perf_df_week_temp.rename(columns=column_rename_mapping_1)
  pincode_perf_df_week_temp['pincode'] = pincode_perf_df_week_temp['pincode'].astype(int).astype(str)
  pincode_df_week = df_shp.merge(pincode_perf_df_week_temp, how='inner', on='pincode')
  pincode_df_week = pincode_df_week.set_index('pincode')


  cluster_perf_df_week_temp = cluster_perf_df_week_temp.rename(columns=column_rename_mapping_2)
  df_cluster_week = pincode_df_week[['Cluster', 'geometry']].dissolve(by='Cluster')
  df_cluster_week.reset_index(inplace = True)
  cluster_df_week = df_cluster_week.set_index('Cluster').join(cluster_perf_df_week_temp.set_index('Cluster'))
#  print(cluster_df_today.head())
#  print(cluster_df_week.head())
  if city == 'Delhi':
    zoom_level = 9.2
  elif city == 'Bangalore':
    zoom_level = 9.5
  elif city == 'Mumbai':
    zoom_level = 10.1
  elif city == 'Hyderabad':
    zoom_level = 9.8
  fig1=plot_pincodes_with_hubs(cluster_df_today, "{} Performance Today".format(city), "OTD", ['OTD', 'OFD', 'FASR'], zoom_level, None)
  fig2=plot_pincodes_with_hubs(cluster_df_week, "{} Performance Last Week".format(city), "OTD", ['OTD', 'OFD', 'FASR'], zoom_level, None)
  fig1.write_html('Clusters/{}_today.html'.format(city))
  print(f'html created for today of {city}')
  fig2.write_html('Clusters/{}_week.html'.format(city))
  print(f'html created for week of {city}')

if SSH_requiered == 'Yes':
  conn.close()
  ssh_tunnel.stop()

  shipping_city cluster_code  shipping_pincode        otd         ofd  \
0     Bangalore      BLDR-07            560103  87.500000  100.000000   
1     Bangalore      BLDR-07            560102  92.424242   98.484848   
2     Bangalore      BNGR-01            560108  33.333333  100.000000   
3     Bangalore      BNGR-01            560083  87.500000  100.000000   
4     Bangalore      BNGR-01            560076  92.592593  100.000000   

        fasr  
0  87.142857  
1  93.750000  
2  33.333333  
3  87.500000  
4  92.592593  
  shipping_city cluster_code        otd    ofd       fasr
0     Bangalore      YSPR-04  82.352941  100.0  82.352941
1     Bangalore      YLHK-05  89.473684  100.0  89.473684
2     Bangalore      WHTF-05  83.529412  100.0  83.950617
3     Bangalore      VDNP-03  88.095238  100.0  88.095238
4     Bangalore      VART-06  89.285714  100.0  89.285714
  shipping_city cluster_code  shipping_pincode        otd         ofd  \
0     Bangalore      BLDR-07            560103  94

In [39]:
!git add .
!git commit -m "Latest update"  
!git push

[main 8c613a4] Latest update
 1 file changed, 5 insertions(+), 5 deletions(-)


To https://github.com/growsimplee/cluster-info.git
   b1f65c0..8c613a4  main -> main
